In [126]:
import pandas as pd
import numpy as np
import time
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import lxml.html
import lxml

import glob

In [2]:
files = glob.glob('output/**/*.csv', recursive=True)
files
    

['output\\debugging.csv',
 'output\\log_error_20170215.csv',
 'output\\log_error_20170221.csv',
 'output\\log_error_20170224.csv',
 'output\\log_error_20170225-1.csv',
 'output\\log_error_20170225.csv',
 'output\\log_opened_20170212.csv',
 'output\\log_opened_20170214.csv',
 'output\\log_opened_20170215.csv',
 'output\\log_opened_20170221.csv',
 'output\\log_opened_20170224.csv',
 'output\\log_opened_20170225-1.csv',
 'output\\log_opened_20170225.csv',
 'output\\scraped_data.csv',
 'output\\scraped_data_20170212.csv',
 'output\\scraped_data_20170214.csv',
 'output\\scraped_data_20170215-1.csv',
 'output\\scraped_data_20170215.csv',
 'output\\scraped_data_20170216.csv',
 'output\\scraped_data_20170221.csv',
 'output\\scraped_data_20170224.csv',
 'output\\scraped_data_20170225.csv',
 'output\\output_1\\log_error_1.csv',
 'output\\output_1\\log_opened_1.csv',
 'output\\output_1\\log_opened_20170222_1.csv',
 'output\\output_1\\log_opened_20170224_1.csv',
 'output\\output_1\\scraped_data_1.

In [3]:
col_names = ['entity', 'department', 'contract', 'year', 'month', 'group', 'lastn', 'lastn2', 'givenn', 'degree', 
             'qualif', 'position', 'region', 'specials', 'currency', 'salary', 'overt', 'start_date', 'end_date', 'obs', 'url']
len(col_names)

21

In [4]:

dfscraped = pd.DataFrame()

for f in files:
    if 'scraped' in f:
        df = pd.read_csv(f, header=None, encoding='latin1', names=col_names, warn_bad_lines=True, low_memory=False)
        df['filename'] = f
        dfscraped = pd.concat([dfscraped, df], ignore_index=True)


In [5]:
dfscraped.filename.value_counts()

output\scraped_data_20170225.csv                 998922
output\output_1\scraped_data_1.csv               360598
output\output_1\scraped_data_20170224_1.csv      193449
output\scraped_data_20170224.csv                  78909
output\output_1\scraped_data_20170223_1.csv       67889
output\scraped_data_20170215.csv                  50586
output\scraped_data_20170214.csv                  44783
output\scraped_data_20170216.csv                  36591
output\scraped_data_20170212.csv                  25227
output\scraped_data_20170221.csv                  15300
output\output_1\old\scraped_data_2_backup.csv      9215
output\output_1\old\scraped_data.csv               8224
output\scraped_data.csv                            4531
output\scraped_data_20170215-1.csv                 3145
output\output_1\old\scraped_data_old.csv           1766
Name: filename, dtype: int64

In [6]:
dfscraped.entity.value_counts()

Ministerio de Justicia                             336990
Ministerio de Vivienda y Urbanismo                 196956
Ministerio de Obras Públicas                       183468
Ministerio de Educación                            128751
Ministerio del Trabajo y Previsión Social          113612
entity                                              92509
Ministerio de Economía, Fomento y Turismo           92159
Ministerio de Desarrollo Social                     84779
Ministerio del Trabajo y PrevisiÃ³n Social          80198
Ministerio de Defensa Nacional                      63995
Ministerio del Interior y Seguridad Pública         53894
Consejo de Defensa del Estado                       52169
b'Ministerio del Interior y Seguridad Pblica'       47911
Ministerio de Agricultura                           44494
b'Ministerio del Trabajo y Previsin Social'         43390
Ministerio del Interior y Seguridad Pblica          40850
Ministerio de MinerÃ­a                              37899
b'Ministerio d

In [33]:
dfscraped.url.nunique()

20597

## Data that was not captured with the right encoding, get again

In [7]:
retake_entity = list(dfscraped[dfscraped.entity.str.contains("b'", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã­", na=False)].entity.value_counts().index)
retake_entity = retake_entity + ['Ministerio del Interior y Seguridad Pblica', 'Presidencia de la Repblica ']  

["b'Ministerio del Interior y Seguridad Pblica'",
 "b'Ministerio del Trabajo y Previsin Social'",
 "b'Ministerio de Obras Pblicas'",
 "b'Ministerio de Educacin'",
 "b'Ministerio de Economa, Fomento y Turismo'",
 "b'Ministerio de Justicia'",
 "b'Ministerio de Desarrollo Social'",
 "b'Ministerio de Defensa Nacional'",
 "b'Ministerio de Salud'",
 "b'Ministerio de Relaciones Exteriores'",
 "b'entity'",
 "b'Ministerio de Transporte y Telecomunicaciones'",
 "b'Ministerio Secretara General de Gobierno'",
 "b'Presidencia de la Repblica'",
 'Ministerio del Trabajo y PrevisiÃ³n Social',
 'Ministerio de MinerÃ\xada',
 'Ministerio de EnergÃ\xada',
 'Consejo Nacional de TelevisiÃ³n',
 'Ministerio de MinerÃ\xada',
 'Ministerio de EnergÃ\xada']

In [8]:
print('Total Urls', dfscraped.url.nunique())
print('Urls to retake', dfscraped[dfscraped.entity.isin(retake_entity)].url.nunique())

Total Urls 20597
Urls to retake 3918


In [9]:
retake_entity_urls = []
for u in list(set(dfscraped.ix[dfscraped.entity.isin(retake_entity), 'url'].unique())):
    if str(u) != 'nan':
        if "b'" in u:
            u = u[2:]
        if u[-1] == "'":
            u = u[:len(u)-1]
        retake_entity_urls.append(u)
len(retake_entity_urls)

3918

In [10]:
retake_entity_urls

['http://www.gobiernotransparentechile.cl/directorio/entidad/8/138/per_contrata/A%C3%B1o%202008/Octubre?x=0&y=0&page_number=3&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/169/per_contrata/Ao-2008?x=0&y=0&page_number=3&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/12/167/per_planta/A%C3%B1o%202013/Octubre',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/11/156/per_planta/Ao-2008',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_contrata/A%C3%B1o%202014/A%C3%B1o%202014%20-%20Mayo?x=0&y=0&page_number=2&sort=id&direction=asc',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/19/271/per_contrata/A%C3%B1o%202014/A%C3%B1o%202014%20-%20Mayo',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/2/9/per_planta/2014/Noviembre',
 'http://www.gobiernotransparentechile.cl/directorio/entidad/4/285/per_planta/2014/Junio',
 'http://www.gobiernotransparentechile.cl

### Open browser and re-scrape data 

In [129]:
output_file = output_file = './output/retake_entity_0.csv'

In [131]:
if not os.path.isfile(output_file):
    browser = webdriver.Firefox()
    for url in retake_entity_urls[0:5]:
        functions.getTableData2(output_file, url, browser)
    

In [65]:
retake_entity_df = pd.read_csv(output_file, encoding='latin1', names=col_names)

In [66]:
retake_entity_df.entity.value_counts()

Ministerio del Trabajo y Previsión Social            75041
b'Ministerio del Trabajo y Previsi\xf3n Social'      50531
Ministerio de Minería                                37565
b'Ministerio de Obras P\xfablicas'                   27766
b'Ministerio del Interior y Seguridad P\xfablica'    25799
b'Ministerio de Educaci\xf3n'                        19799
b'Ministerio de Econom\xeda, Fomento y Turismo'      14122
b'Ministerio de Justicia'                            12252
b'Ministerio de Desarrollo Social'                    9425
b'Ministerio de Defensa Nacional'                     4552
Ministerio de Energía                                 4545
Ministerio de Desarrollo Social                       4113
Ministerio de Educación                               3450
Ministerio de Justicia                                3168
Ministerio de Economía, Fomento y Turismo             3124
b'Ministerio de Salud'                                3089
Ministerio del Interior y Seguridad Pública           22

In [ ]:
cleanLatin(retake_entity_df)

### Let's delete the corresponding rows and add the new ones 

In [118]:
retake_entity_df['filename'] = 'retake_entity_0.csv'
print(retake_entity_df.shape)
print(dfscraped[dfscraped.entity.isin(retake_entity)].shape)
dfscraped = dfscraped[~dfscraped.entity.isin(retake_entity)]
dfscraped = pd.concat([dfscraped, retake_entity_df])
dfscraped.shape

(308555, 22)
(310284, 22)


In [120]:
dfscraped.entity.value_counts()

Ministerio de Justicia                             352410
Ministerio del Trabajo y Previsión Social          239184
Ministerio de Obras Públicas                       211615
Ministerio de Vivienda y Urbanismo                 196956
Ministerio de Educación                            152000
Ministerio de Economía, Fomento y Turismo          109405
Ministerio de Desarrollo Social                     98317
entity                                              92509
Ministerio del Interior y Seguridad Pública         81918
Ministerio de Defensa Nacional                      69906
Consejo de Defensa del Estado                       52169
Ministerio de Agricultura                           44494
Ministerio del Interior y Seguridad Pblica          40850
Ministerio de Minería                               38195
Presidencia de la República                         27483
Ministerio de Relaciones Exteriores                 23776
Ministerio Secretaría General de la Presidencia     13534
presidencia_de

### Two entities that signal missing vowels 

In [158]:
retake_entity = list(dfscraped[dfscraped.entity.str.contains("b'", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã­", na=False)].entity.value_counts().index)
retake_entity = retake_entity + ['Ministerio del Interior y Seguridad Pblica', 'Presidencia de la Repblica']  
print retake_entity
print('Total Urls', dfscraped.url.nunique())
print('Urls to retake', dfscraped[dfscraped.entity.isin(retake_entity)].url.nunique())
retake_entity_urls = []
for u in list(set(dfscraped.ix[dfscraped.entity.isin(retake_entity), 'url'].unique())):
    if str(u) != 'nan':
        if "b'" in u:
            u = u[2:]
        if u[-1] == "'":
            u = u[:len(u)-1]
        retake_entity_urls.append(u)
len(retake_entity_urls)

['Ministerio del Interior y Seguridad Pblica', 'Presidencia de la Repblica']

Total Urls 19208
Urls to retake 445


445

In [154]:
output_file = output_file = './output/retake_entity_1.csv'

In [155]:
if not os.path.isfile(output_file):
    browser = webdriver.Firefox()
    for url in tqdm_notebook(retake_entity_urls):
        functions.getTableData2(output_file, url, browser)
    

In [156]:
retake_entity_df = pd.read_csv(output_file, encoding='latin1', names=col_names)
retake_entity_df.entity.value_counts()

b'Ministerio del Interior y Seguridad P\xfablica'    19800
Ministerio del Interior y Seguridad Pública            258
b'Presidencia de la Rep\xfablica'                      142
Presidencia de la República                             89
Name: entity, dtype: int64

In [157]:
cleanLatin(retake_entity_df)
retake_entity_df.entity.value_counts()

Ministerio del Interior y Seguridad Pública    20058
Presidencia de la República                      231
Name: entity, dtype: int64

In [161]:
retake_entity_df['filename'] = 'retake_entity_1.csv'
print(retake_entity_df.shape)
print(dfscraped[dfscraped.entity.isin(retake_entity)].shape)
dfscraped = dfscraped[~dfscraped.entity.isin(retake_entity)]
dfscraped = pd.concat([dfscraped, retake_entity_df])
dfscraped.shape

(20289, 22)
(40992, 22)


(1876703, 22)

In [163]:
dfscraped.entity.value_counts()

Ministerio de Justicia                             352410
Ministerio del Trabajo y Previsión Social          239184
Ministerio de Obras Públicas                       211615
Ministerio de Vivienda y Urbanismo                 196956
Ministerio de Educación                            152000
Ministerio de Economía, Fomento y Turismo          109405
Ministerio del Interior y Seguridad Pública        101976
Ministerio de Desarrollo Social                     98317
entity                                              92509
Ministerio de Defensa Nacional                      69906
Consejo de Defensa del Estado                       52169
Ministerio de Agricultura                           44494
Ministerio de Minería                               38195
Presidencia de la República                         27714
Ministerio de Relaciones Exteriores                 23776
Ministerio Secretaría General de la Presidencia     13534
presidencia_de_la_republica                          9215
Ministerio de 

In [167]:
dfscraped.ix[(dfscraped.entity == 'entity') & (dfscraped.url.str.contains('FUNCIONARIO')), 'url']

1161960     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1161962     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1161964     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1162023     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1162079     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1162085     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1162143     - FUNCIONARIO PERCIBE ASIGNACIÓN DE FUNCIÓN C...
1162145     - FUNCIONARIO PERCIBE ASIGNACIÓN DE FUNCIÓN C...
1162146     - FUNCIONARIO PERCIBE ASIGNACIÓN DE FUNCIÓN C...
1162150     - FUNCIONARIO PERCIBE ASIGNACIÓN DE FUNCIÓN C...
1162162     - FUNCIONARIO SE ENCUENTRA EN COMISIÓN DE SER...
1167886     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1167887     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1167888     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1167889     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1167890     - FUNCIONARIO PERCIBE ASIGNACION DE FUNCION C...
1167891     - FUNCIONARI

In [168]:
dfscraped.ix[1161960, 'url']

'\xa0- FUNCIONARIO PERCIBE ASIGNACION DE FUNCION CRITICA DE UN:39.1%'

In [195]:
# Clean Salary
dfscraped.salary = dfscraped.salary.str.replace('.', '')

# Drop duplicates and write new
dfscraped_clean = dfscraped.drop_duplicates()

# Write clean file 

In [170]:

print(dfscraped.shape)
print(dfscraped_clean.shape)
dfscraped_clean.to_csv('./output/scraped_data_clean.csv')

(1876703, 22)
(1815944, 22)


# Some stats 

In [204]:
# Be able to get first stats in Ministerio de Justicia